In [1]:
%cd ../src

/Users/shanekercheval/repos/sik-llms/src


# Clients

In [2]:
# For "registered" clients (via `@Client.register`), the client
# can be created with `create_client` by passing in the model name.
from sik_llms import create_client

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [3]:
# Or, the client can be directly instantiated
from sik_llms import OpenAI
client = OpenAI(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [4]:
# Or, the client can be directly instantiated
from sik_llms import Anthropic
client = Anthropic(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
client

# Chat

In [3]:
from sik_llms import create_client, user_message, TextChunkEvent

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)

message = user_message("What is the capital of France?")
message

{'role': 'user', 'content': 'What is the capital of France?'}

### Run Synchronously via `__call__`

In [6]:
response = client(messages=[message])
print(response)
print(response.response)

input_tokens=14 output_tokens=8 input_cost=2.1e-06 output_cost=4.8e-06 cache_write_tokens=None cache_read_tokens=0 cache_write_cost=None cache_read_cost=0.0 duration_seconds=1.123208999633789 response='The capital of France is Paris.'
The capital of France is Paris.


### Run Aynchronously via `run_async`

In [7]:
response = await client.run_async(messages=[message])
print(response)
print(response.response)

input_tokens=14 output_tokens=8 input_cost=2.1e-06 output_cost=4.8e-06 cache_write_tokens=None cache_read_tokens=0 cache_write_cost=None cache_read_cost=0.0 duration_seconds=0.8110058307647705 response='The capital of France is Paris.'
The capital of France is Paris.


### Stream Asynchronously via `stream`

In [10]:
responses = []
async for response in client.stream(messages=[message]):
    if isinstance(response, TextChunkEvent):
        print(response.content, end="")
    responses.append(response)

The capital of France is Paris.

In [11]:
responses

[TextChunkEvent(content='The', logprob=None),
 TextChunkEvent(content=' capital', logprob=None),
 TextChunkEvent(content=' of', logprob=None),
 TextChunkEvent(content=' France', logprob=None),
 TextChunkEvent(content=' is', logprob=None),
 TextChunkEvent(content=' Paris', logprob=None),
 TextChunkEvent(content='.', logprob=None),
 TextResponse(input_tokens=14, output_tokens=8, input_cost=2.1e-06, output_cost=4.8e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=0.4176782909780741, response='The capital of France is Paris.')]

### Generate multiple responses from a single input via `sample`

In [12]:
client = create_client(model_name='gpt-4o-mini', temperature=1.5)
responses = await client.sample(
    messages=[user_message('Generate a random number. Return only the number')],
    n=3,
)
print([r.response for r in responses])
responses

['42', '47829', '74']


[TextResponse(input_tokens=16, output_tokens=2, input_cost=2.4e-06, output_cost=1.2e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=0.5277789169922471, response='42'),
 TextResponse(input_tokens=16, output_tokens=3, input_cost=2.4e-06, output_cost=1.8e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=0.6055014161393046, response='47829'),
 TextResponse(input_tokens=16, output_tokens=2, input_cost=2.4e-06, output_cost=1.2e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=0.45302558317780495, response='74')]

### Generate multiple responses from multiple inputs via `generate_multiple`

In [5]:
client = create_client(model_name='gpt-4o-mini', temperature=1.5)
responses = await client.generate_multiple(
    messages=[
        [user_message("What is the capital of France? Return only the city name.")],
        [user_message("What is the capital of Italy? Return only the city name.")],
    ],
)
print([r.response for r in responses])
responses

['Paris', 'Rome']


[TextResponse(input_tokens=20, output_tokens=2, input_cost=3e-06, output_cost=1.2e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=0.8573106250260025, response='Paris'),
 TextResponse(input_tokens=20, output_tokens=2, input_cost=3e-06, output_cost=1.2e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=0.8546491248998791, response='Rome')]

### Generate `sample_n` responses from multiple inputs via `generate_multiple`

In [6]:
client = create_client(model_name='gpt-4o-mini', temperature=1.5)
responses_set = await client.generate_multiple(
    messages=[
        [user_message("Pick a random number between 1 and 100. Return only the number.")],
        [user_message("Pick a random number between 100 and 200. Return only the number.")],
    ],
    sample_n=5,
)
for responses in responses_set:
    print([r.response for r in responses])
responses_set

['47', '43', '47', '42', '67']
['137', '156', '154', '147', '153']


[[TextResponse(input_tokens=23, output_tokens=2, input_cost=3.45e-06, output_cost=1.2e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=1.2576175420545042, response='47'),
  TextResponse(input_tokens=23, output_tokens=2, input_cost=3.45e-06, output_cost=1.2e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=0.8492839590180665, response='43'),
  TextResponse(input_tokens=23, output_tokens=2, input_cost=3.45e-06, output_cost=1.2e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=0.8483560001477599, response='47'),
  TextResponse(input_tokens=23, output_tokens=2, input_cost=3.45e-06, output_cost=1.2e-06, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=1.1479352081660181, response='42'),
  TextResponse(input_tokens=23, output_tokens=2, input_cost=3.45

---

# OpenAI Functions/Tools

In [11]:
from sik_llms import (
    create_client, user_message,
    Tool, Parameter, RegisteredClients,
)

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            param_type=str,
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.OPENAI_TOOLS,
    model_name='gpt-4o-mini',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=60 output_tokens=17 input_cost=9e-06 output_cost=1.0199999999999999e-05 cache_write_tokens=None cache_read_tokens=0 cache_write_cost=None cache_read_cost=0.0 duration_seconds=0.8125739097595215 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='call_fiQ9qpvKIxfucsKHigT3nr2O') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='call_fiQ9qpvKIxfucsKHigT3nr2O'


---

# Claude Functions/Tools

In [12]:
from sik_llms import (
    create_client, user_message,
    Tool, Parameter, RegisteredClients,
)

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            param_type=str,
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.ANTHROPIC_TOOLS,
    model_name='claude-3-7-sonnet-latest',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=401 output_tokens=40 input_cost=0.001203 output_cost=0.0006000000000000001 cache_write_tokens=0 cache_read_tokens=0 cache_write_cost=0.0 cache_read_cost=0.0 duration_seconds=1.5602500438690186 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='toolu_019xogKTwcVzGRxdVP3Xamof') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='toolu_019xogKTwcVzGRxdVP3Xamof'


---

# Structured Outputs via OpenAI

In [ ]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='gpt-4o-mini',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = await client.run_async(messages=messages)
# or `response = client(messages=messages)` for synchronous execution
print(response)
print('---')
print(response.parsed)

input_tokens=92 output_tokens=18 input_cost=1.38e-05 output_cost=1.08e-05 cache_write_tokens=None cache_read_tokens=0 cache_write_cost=None cache_read_cost=0.0 duration_seconds=0.8882229158189148 parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']) refusal=None
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Structured Outputs via Anthropic

In [14]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = await client.run_async(messages=messages)
# or `response = client(messages=messages)` for synchronous execution
print(response)
print('---')
print(response.parsed)

input_tokens=441 output_tokens=78 input_cost=0.001323 output_cost=0.00117 cache_write_tokens=None cache_read_tokens=None cache_write_cost=None cache_read_cost=None duration_seconds=2.373826026916504 parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']) refusal=None
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Reasoning via OpenAI

In [15]:
from sik_llms import (
    create_client, user_message,
    TextChunkEvent, TextResponse, ReasoningEffort,
)

client = create_client(
    model_name='o3-mini',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None
async for response in client.stream(messages=messages):
    if isinstance(response, TextChunkEvent):
        print(response.content, end="")
    elif isinstance(response, TextResponse):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")

To solve the expression 1 + 2 + (3 * 4) + (5 * 6), follow these steps:

1. First, calculate the multiplication parts:
   - 3 * 4 = 12
   - 5 * 6 = 30

2. Then, add the results along with the remaining numbers:
   - 1 + 2 = 3
   - Now, 3 + 12 = 15
   - Finally, 15 + 30 = 45

So, the final answer is 45.

In [16]:
summary

TextResponse(input_tokens=27, output_tokens=193, input_cost=2.97e-05, output_cost=0.0008492, cache_write_tokens=None, cache_read_tokens=0, cache_write_cost=None, cache_read_cost=0.0, duration_seconds=3.037583827972412, response='To solve the expression 1 + 2 + (3 * 4) + (5 * 6), follow these steps:\n\n1. First, calculate the multiplication parts:\n   - 3 * 4 = 12\n   - 5 * 6 = 30\n\n2. Then, add the results along with the remaining numbers:\n   - 1 + 2 = 3\n   - Now, 3 + 12 = 15\n   - Finally, 15 + 30 = 45\n\nSo, the final answer is 45.')

---

# Reasoning via Claude

In [17]:
from sik_llms import (
    create_client, user_message,
    TextChunkEvent, ThinkingChunkEvent,
    TextResponse, ReasoningEffort,
)

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None

current_type = None
async for response in client.stream(messages=messages):
    is_text_chunk = isinstance(response, TextChunkEvent)
    is_thinking_chunk = isinstance(response, ThinkingChunkEvent)
    is_summary = isinstance(response, TextResponse)

    if is_text_chunk or is_thinking_chunk:
        if type(response) is not current_type:
            print(f"\n\n[{'THINKING' if is_thinking_chunk else 'TEXT'}]")
            current_type = type(response)
        print(response.content, end="")
    elif isinstance(response, TextResponse):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")



[THINKING]
I need to calculate the value of the expression 1 + 2 + (3 * 4) + (5 * 6).

Following the order of operations (PEMDAS), I need to do the multiplication first, then addition.

(3 * 4) = 12
(5 * 6) = 30

Now I have:
1 + 2 + 12 + 30

Adding these up:
1 + 2 = 3
3 + 12 = 15
15 + 30 = 45

So the answer is 45.

[TEXT]
To solve this expression, I'll follow the order of operations (PEMDAS):

First, calculate the expressions in parentheses:
- (3 * 4) = 12
- (5 * 6) = 30

Then add all terms:
1 + 2 + 12 + 30 = 45

The answer is 45.

In [18]:
summary

TextResponse(input_tokens=60, output_tokens=235, input_cost=0.00018, output_cost=0.003525, cache_write_tokens=0, cache_read_tokens=0, cache_write_cost=0.0, cache_read_cost=0.0, duration_seconds=3.8085758686065674, response="I need to calculate the value of the expression 1 + 2 + (3 * 4) + (5 * 6).\n\nFollowing the order of operations (PEMDAS), I need to do the multiplication first, then addition.\n\n(3 * 4) = 12\n(5 * 6) = 30\n\nNow I have:\n1 + 2 + 12 + 30\n\nAdding these up:\n1 + 2 = 3\n3 + 12 = 15\n15 + 30 = 45\n\nSo the answer is 45.To solve this expression, I'll follow the order of operations (PEMDAS):\n\nFirst, calculate the expressions in parentheses:\n- (3 * 4) = 12\n- (5 * 6) = 30\n\nThen add all terms:\n1 + 2 + 12 + 30 = 45\n\nThe answer is 45.")

---

# ReasoningAgent

In [19]:
import json
from sik_llms.models_base import (
    Tool, Parameter, ThinkingEvent, ToolPredictionEvent,
    ToolResultEvent, TextChunkEvent, ErrorEvent, TextResponse,
)
from sik_llms.reasoning_agent import ReasoningAgent

####
# Define the tool functions
####
async def calculator(expression: str) -> str:
    """Execute calculator tool."""
    try:
        # Only allow simple arithmetic for safety
        allowed_chars = set('0123456789+-*/() .')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e!s}"


async def weather(location: str, units: str) -> str:
    """Mock weather tool - returns fake data."""
    # Return mock weather data
    weather_data = {
        'New York': '68',
        'San Francisco': '62',
        'Miami': '85',
        'Chicago': '55',
        'Los Angeles': '75',
    }
    for city in weather_data:  # noqa: PLC0206
        if city.lower() in location.lower():
            temp = weather_data[city]
            if units == 'C':
                # C = (°F - 32) x (5/9)
                temp = round((temp - 32) * 5 / 9)
            return {location: f"{temp}°{units}"}
    return None

####
# Define tool objects
####
calculator_tool = Tool(
    name='calculator',
    description="Perform mathematical calculations",
    parameters=[
        Parameter(
            name='expression',
            param_type=str,
            required=True,
            description="The mathematical expression to evaluate (e.g., '2 + 2', '5 * 10')",
        ),
    ],
    func=calculator,
)

weather_tool = Tool(
    name="get_weather",
    description="Get the current weather for a location",
    parameters=[
        Parameter(
            name="location",
            param_type=str,
            required=True,
            description="The name of the city (e.g., 'San Francisco', 'New York', 'London')",
        ),
        Parameter(
            name='units',
            param_type=str,
            required=True,
            description="The units for temperature",
            valid_values=['F', 'C'],
        ),
    ],
    func=weather,
)

# Create the reasoning agent
agent = ReasoningAgent(
    model_name="gpt-4o-mini",  # You can change this to other models
    tools=[calculator_tool, weather_tool],
    max_iterations=10,
    temperature=0,
)

question = "I'm planning a trip to New York and Miami. What's the weather like in both cities? Also, if I have a budget of $2400 for a 6-day trip, how much can I spend per day?"  # noqa: E501
# Run the agent and collect the results
messages = [{"role": "user", "content": question}]

print(f"[QUESTION]: {question}\n")

current_iteration = 0

async for event in agent.stream(messages):
    if isinstance(event, ThinkingEvent):
        if hasattr(event, 'iteration') and event.iteration != current_iteration:
            current_iteration = event.iteration
            print(f"\n--- Iteration {current_iteration}\n")
        if event.content:
            print(f"\n[THINKING]:\n{event.content}")

    elif isinstance(event, ToolPredictionEvent):
        print("\n[TOOL PREDICTION]:")
        print(f"Tool: {event.name}`")
        print(f"Parameters: \n```json\n{json.dumps(event.arguments, indent=2)}\n```")

    elif isinstance(event, ToolResultEvent):
        print("\n[TOOL RESULT]:")
        print(f"Tool: {event.name}`")
        print(f"Result: {event.result}")

    elif isinstance(event, ErrorEvent):
        print("\n[ERROR]:")
        print(f"Error: {event.content}")

    elif isinstance(event, TextChunkEvent):
        # For streaming text generation
        if current_iteration  >= 0:  # Only print once for the header
            print("\n[FINAL RESPONSE]:")
            current_iteration = -1  # Set to an impossible iteration to avoid repeating

        print(event.content, end="")

    elif isinstance(event, TextResponse):
        # Print nothing here as we've already streamed the response
        pass

# Just for notebook display cleanliness
print("\n\n---\n\n")

[QUESTION]: I'm planning a trip to New York and Miami. What's the weather like in both cities? Also, if I have a budget of $2400 for a 6-day trip, how much can I spend per day?


--- Iteration 1


[THINKING]:
To provide the weather information, I need to check the current weather for both New York and Miami. Additionally, I need to calculate the daily budget for a 6-day trip with a total budget of $2400. First, I will calculate the daily budget, and then I will check the weather for both cities.

[TOOL PREDICTION]:
Tool: get_weather`
Parameters: 
```json
{
  "location": "New York",
  "units": "F"
}
```

[TOOL RESULT]:
Tool: get_weather`
Result: {'New York': '68°F'}

--- Iteration 2


[THINKING]:
I have obtained the current weather for New York, which is 68°F. Now, I will proceed to get the current weather for Miami. After that, I will calculate the daily budget for the trip.

[TOOL PREDICTION]:
Tool: get_weather`
Parameters: 
```json
{
  "location": "Miami",
  "units": "F"
}
```

[TOOL

In [20]:
print(f"Input Tokens: {event.input_tokens}")
print(f"Output Tokens: {event.output_tokens}")
print(f"Total Cost: {event.total_cost}")
print(f"Duration: {event.duration_seconds:.2f} seconds")

Input Tokens: 5013
Output Tokens: 503
Total Cost: 0.00105375
Duration: 10.58 seconds


---

# Prompt Caching

## Anthropic - `cache_control` parameter in `system_message`

In [21]:
from faker import Faker

cache_content = Faker().text(max_nb_chars=15_000)
cache_content[0:200]

'Mean great page four learn. Mr type others gas perhaps. Near least leg say.\nCould plan station foot material bring. Individual live answer scene. Catch federal theory exist for citizen on recent. Resp'

In [22]:
####
# This example is modified from anthropic's prompt-caching.ipynb notebook
# https://github.com/anthropics/anthropic-cookbook/blob/main/misc/prompt_caching.ipynb
####
from sik_llms import Anthropic
from sik_llms.models_base import system_message, user_message, assistant_message

client = Anthropic(
    model_name='claude-3-5-haiku-latest',
    temperature=0.1,
)
# https://github.com/anthropics/anthropic-cookbook/blob/main/misc/prompt_caching.ipynb
system_messages = [
    system_message("You are a helpful assistant."),
    system_message(
        cache_content,
        cache_control={'type': 'ephemeral'},
    ),
]
messages = [
    *system_messages,
    user_message("What is the first word of the cached text?"),
]

# first run should result in a cache-miss & write
response = await client.run_async(messages=messages)
print(response.response)
print('---')
print(f"Total Cost: {response.total_cost}")
print('---')
print(f"Input Tokens: {response.input_tokens}")
print(f"Output Tokens: {response.output_tokens}")
print(f"Cache Write Tokens: {response.cache_write_tokens}")
print(f"Cache Read Tokens: {response.cache_read_tokens}")
print(f"Total Tokens: {response.total_tokens}")

The first word of the cached text is "Mean".
---
Total Cost: 0.0029235999999999997
---
Input Tokens: 17
Output Tokens: 14
Cache Write Tokens: 2854
Cache Read Tokens: 0
Total Tokens: 2885


In [23]:
# second run should result in a cache-hit & read
messages = [
    *system_messages,
    user_message("What is the first word of the cached text?"),
    assistant_message(response.response),
    user_message("What is the second word of the cached text?"),
]
response = await client.run_async(messages=messages)
print(response.response)
print('---')
print(f"Total Cost: {response.total_cost}")
print('---')
print(f"Input Tokens: {response.input_tokens}")
print(f"Output Tokens: {response.output_tokens}")
print(f"Cache Write Tokens: {response.cache_write_tokens}")
print(f"Cache Read Tokens: {response.cache_read_tokens}")
print(f"Total Tokens: {response.total_tokens}")

The second word of the cached text is "great".
---
Total Cost: 0.00031952
---
Input Tokens: 44
Output Tokens: 14
Cache Write Tokens: 0
Cache Read Tokens: 2854
Total Tokens: 2912


## Anthropic - `cache_content` parameter in `__init__`

In [24]:
from faker import Faker

cache_content = Faker().text(max_nb_chars=15_000)
cache_content[0:200]

'Home attention behavior agent plan. Writer stock direction beyond entire cup free.\nShe mother floor fact reason hope culture. Way put card life someone best example interest. Traditional behavior over'

In [25]:
####
# This example is modified from anthropic's prompt-caching.ipynb notebook
# https://github.com/anthropics/anthropic-cookbook/blob/main/misc/prompt_caching.ipynb
####
from sik_llms import Anthropic
from sik_llms.models_base import system_message, user_message, assistant_message

client = Anthropic(
    model_name='claude-3-5-haiku-latest',
    temperature=0.1,
    cache_content=cache_content,
)
# https://github.com/anthropics/anthropic-cookbook/blob/main/misc/prompt_caching.ipynb
messages = [
    system_message("You are a helpful assistant."),
    user_message("What is the first word of the cached text?"),
]

# first run should result in a cache-miss & write
response = await client.run_async(messages=messages)
print(response.response)
print('---')
print(f"Total Cost: {response.total_cost}")
print('---')
print(f"Input Tokens: {response.input_tokens}")
print(f"Output Tokens: {response.output_tokens}")
print(f"Cache Write Tokens: {response.cache_write_tokens}")
print(f"Cache Read Tokens: {response.cache_read_tokens}")
print(f"Total Tokens: {response.total_tokens}")

The first word of the cached text is "Home".
---
Total Cost: 0.0029235999999999997
---
Input Tokens: 17
Output Tokens: 14
Cache Write Tokens: 2854
Cache Read Tokens: 0
Total Tokens: 2885


In [26]:
# second run should result in a cache-hit & read
messages = [
    system_message("You are a helpful assistant."),
    user_message("What is the first word of the cached text?"),
    assistant_message(response.response),
    user_message("What is the second word of the cached text?"),
]
response = await client.run_async(messages=messages)
print(response.response)
print('---')
print(f"Total Cost: {response.total_cost}")
print('---')
print(f"Input Tokens: {response.input_tokens}")
print(f"Output Tokens: {response.output_tokens}")
print(f"Cache Write Tokens: {response.cache_write_tokens}")
print(f"Cache Read Tokens: {response.cache_read_tokens}")
print(f"Total Tokens: {response.total_tokens}")

The second word of the cached text is "attention".
---
Total Cost: 0.00031952
---
Input Tokens: 44
Output Tokens: 14
Cache Write Tokens: 0
Cache Read Tokens: 2854
Total Tokens: 2912


---

# Misc Examples

## Bedrock via OpenAI API

In [ ]:
import os
from sik_llms import OpenAI, user_message
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    server_url=os.getenv('BEDROCK_API_URL'),
    api_key=os.getenv('BEDROCK_API_KEY'),
    model_name='anthropic.claude-3-haiku-20240307-v1:0',
    user='bedrock-requires-user?',
)
response = client(messages=[user_message("What is the capital of France?")])
print(response.response)

---

## OpenAI Log Probs

In [29]:
import math
from sik_llms import OpenAI, user_message, TextChunkEvent
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    model_name='gpt-4o-mini',
    logprobs=True,
)
messages = [
    user_message("Write a haiku about the ocean."),
]
async for response in client.stream(messages=messages):
    if isinstance(response, TextChunkEvent):
        log_prob = response.logprob
        prob = math.exp(log_prob)
        print(f"{response.content} (logprob: {log_prob:.2f}, prob: {prob:.2f})")

End (logprob: -1.61, prob: 0.20)
less (logprob: -0.00, prob: 1.00)
 waves (logprob: -0.03, prob: 0.97)
 whisper (logprob: -0.12, prob: 0.88)
, (logprob: -0.00, prob: 1.00)
  
 (logprob: -0.00, prob: 1.00)
Secrets (logprob: -0.86, prob: 0.43)
 of (logprob: -0.06, prob: 0.94)
 the (logprob: -0.02, prob: 0.98)
 deep (logprob: -0.00, prob: 1.00)
 blue (logprob: -1.06, prob: 0.35)
 world (logprob: -1.08, prob: 0.34)
, (logprob: -0.04, prob: 0.96)
  
 (logprob: -0.00, prob: 1.00)
Salt (logprob: -1.22, prob: 0.30)
 and (logprob: -1.60, prob: 0.20)
 sun (logprob: -0.29, prob: 0.75)
 embrace (logprob: -0.70, prob: 0.50)
. (logprob: -0.00, prob: 1.00)


---

## Images

In [30]:
from sik_llms import create_client, user_message, ImageContent
from dotenv import load_dotenv
load_dotenv()

image = ImageContent.from_url(
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/320px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
)

client = create_client(model_name='gpt-4o-mini')
response = client(messages=[
    user_message([
        "What's in this image? Describe it briefly.",
        image,
    ]),
])
print(response)
print('---')
print(response.response)

input_tokens=8516 output_tokens=53 input_cost=0.0012774 output_cost=3.18e-05 cache_write_tokens=None cache_read_tokens=0 cache_write_cost=None cache_read_cost=0.0 duration_seconds=1.5114619731903076 response='The image features a wooden boardwalk or pathway leading through a grassy landscape. The path extends into a wide-open area covered with tall green grass, surrounded by trees and bushes in the background. The sky above is bright with fluffy clouds, suggesting a pleasant day.'
---
The image features a wooden boardwalk or pathway leading through a grassy landscape. The path extends into a wide-open area covered with tall green grass, surrounded by trees and bushes in the background. The sky above is bright with fluffy clouds, suggesting a pleasant day.


---

## Web Search w/ Claude

In [2]:
from sik_llms import create_client, user_message
from dotenv import load_dotenv
load_dotenv()

client = create_client(model_name='claude-3-7-sonnet-latest', web_search=True)
response = client(messages=[user_message("What the current net worth of Michael Jordan?")])
print(response)
print('---')
print(response.response)

input_tokens=2684 output_tokens=1034 input_cost=0.008052 output_cost=0.015510000000000001 cache_write_tokens=0 cache_read_tokens=0 cache_write_cost=0.0 cache_read_cost=0.0 duration_seconds=47.10643941699527 response='I\'ll help you find the current net worth of Michael Jordan. Let me search for the most up-to-date information.\n\nBased on the search results, I can provide you with information about Michael Jordan\'s current net worth in 2025.\n\nMichael Jordan has an estimated net worth of $3.5 billion in 2025. This makes him one of the wealthiest former athletes in the world.\n\n## Sources of His Wealth\n\nJordan\'s net worth consists of earnings from his successful NBA career, lucrative brand endorsements, and successful business ventures. Here\'s a breakdown of his major income sources:\n\n1. **NBA Career Earnings**:\nAccording to Forbes, Jordan\'s net worth makes him the richest former professional athlete in the world. He became the first billionaire athlete and joined the "three-

---